## Ансамбли и полносвязные нейронные сети
В этом ноутбуке вам нужно обучить модели на датасете классификации из предыдущего ноутбука и сравнить результаты. Вам будет предоставлен baseline, на основе которого вы будете доделывать предсказывающие модели. Оценка лабы будет зависеть от ROC-AUC на тестовых данных по следующим критериям:
\
AUC - на тестовых данных
- $AUC \leq 0.76$ - 0 баллов
- $0.76 < AUC \leq 0.77$ - 2 балла
- $0.77 < AUC \leq 0.78$ - 4 балла
- $0.78 < AUC \leq 0.79$ - 6 баллов
- $0.79 < AUC \leq 0.80$ - 8 баллов
- $AUC > 0.80$ - 10 баллов


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import roc_auc_score, precision_score, recall_score, roc_curve, accuracy_score

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('german.csv', sep=';')
print(data.head())

X = data.iloc[:, 1:].to_numpy()
y = data.iloc[:, 0].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
plt.hist(y_train, bins=2, edgecolor='k')
plt.xticks([0, 1])
plt.xlabel('Class (0: Non-Creditworthy, 1: Creditworthy)')
plt.ylabel('Count')
plt.title('Distribution of Classes in Training Data')
plt.show()

In [ ]:
from sklearn.utils.discovery import all_estimators
print(*all_estimators(type_filter="classifier"))

In [ ]:
import itertools
from IPython.display import clear_output

param_grid = {
    'criterion' : ['entropy'],
    'n_estimators': range(10,13),
    'class_weight': ['balanced', 'balanced_subsample'],
    'max_depth': range(3,5),
    'min_samples_split': range(2,10),
    'max_leaf_nodes': range(10,20),
}


combinations = list(itertools.product(*param_grid.values()))
bestParams = ''
bestScore = 0

for combination in combinations:
    params = dict(zip(param_grid.keys(), combination))

    
    
    rf_model = RandomForestClassifier(n_estimators=params['n_estimators'], criterion=params['criterion'], class_weight=params['class_weight'], max_depth=params['max_depth'],min_samples_split=params['min_samples_split'],max_leaf_nodes=params['max_leaf_nodes'],random_state=42)
    rf_model.fit(X_train, y_train)

    rf_pred = rf_model.predict(X_test)
    rf_roc_auc = roc_auc_score(y_test, rf_pred)
    print(params)
    print(rf_roc_auc)
    if (rf_roc_auc > bestScore): 
        bestParams = params
        bestScore = rf_roc_auc
print()
print(bestParams)
print(bestScore)

In [ ]:
import itertools
from IPython.display import clear_output

param_grid = {
    'learning_rate': [0.005, 0.01, 0.02, 0.05, 0.1, 0.2],
    'n_estimators' : range(10,110,10),
    'min_weight_fraction_leaf': [i/1000 for i in range(1,300)],
    'max_depth':[2],#range(2,10),
    'max_features':[9],#range(6,10),
    'subsample':[0.8, 0.9]
}

combinations = list(itertools.product(*param_grid.values()))
bestParams = ''
bestScore = 0

for combination in combinations:
    params = dict(zip(param_grid.keys(), combination))

    gb_model = GradientBoostingClassifier(learning_rate=params['learning_rate'],n_estimators=params['n_estimators'],min_weight_fraction_leaf=params['min_weight_fraction_leaf'],min_samples_split=params['min_samples_split'],min_samples_leaf=params['min_samples_leaf'],subsample=params['subsample'], max_features=params['max_features'], max_depth=params['max_depth'], random_state=42)
    gb_model.fit(X_train, y_train)

    gb_pred = gb_model.predict_proba(X_test)[:, 1]
    gb_roc_auc = roc_auc_score(y_test, gb_pred)
    print(params)
    print(gb_roc_auc)
    if (gb_roc_auc > bestScore): 
        bestParams = params
        bestScore = gb_roc_auc
print()
print(bestParams)
print(bestScore)

In [291]:
# Обучение Random Forest
rf_model = RandomForestClassifier(criterion='entropy', class_weight='balanced_subsample', max_depth=4, n_estimators=11,random_state=42)
rf_model.fit(X_train, y_train)

# Прогноз на тестовых данных
rf_pred = rf_model.predict(X_test)
rf_pred_prob = rf_model.predict_proba(X_test)[:, 1]

# Расчет метрик для Random Forest
rf_roc_auc = roc_auc_score(y_test, rf_pred_prob)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred)
rf_recall = recall_score(y_test, rf_pred)

print("Random Forest метрики:")
print(f"ROC AUC: {rf_roc_auc:.2f}")
print(f"Accuracy: {rf_accuracy:.2f}")
print(f"Precision: {rf_precision:.2f}")
print(f"Recall: {rf_recall:.2f}")

# Обучение Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=40, learning_rate=0.2,min_weight_fraction_leaf=0.135,max_depth=2, max_features=9, subsample=0.8, random_state=42)
gb_model.fit(X_train, y_train)

# Прогноз на тестовых данных
gb_pred = gb_model.predict(X_test)
gb_pred_prob = gb_model.predict_proba(X_test)[:, 1]

# Расчет метрик для Gradient Boosting
gb_roc_auc = roc_auc_score(y_test, gb_pred_prob)
gb_accuracy = accuracy_score(y_test, gb_pred)
gb_precision = precision_score(y_test, gb_pred)
gb_recall = recall_score(y_test, gb_pred)

print("\nGradient Boosting метрики:")
print(f"ROC AUC: {gb_roc_auc:.2f}")
print(f"Accuracy: {gb_accuracy:.2f}")
print(f"Precision: {gb_precision:.2f}")
print(f"Recall: {gb_recall:.2f}")

Random Forest метрики:
ROC AUC: 0.82
Accuracy: 0.79
Precision: 0.89
Recall: 0.79

Gradient Boosting метрики:
ROC AUC: 0.81
Accuracy: 0.77
Precision: 0.79
Recall: 0.91


In [ ]:
import itertools
from IPython.display import clear_output

param_grid = {
    'solver': ['lbfgs'],
    'max_iter': [900],
    'alpha': [10.0 ** -i for i in range(6, 10)],
    'hidden_layer_sizes':np.arange(2, 30)
}

combinations = list(itertools.product(*param_grid.values()))
bestParams = ''
bestScore = 0

for combination in combinations:
    params = dict(zip(param_grid.keys(), combination))

    gb_model = MLPClassifier(solver=params['solver'],max_iter=params['max_iter'],alpha=params['alpha'],hidden_layer_sizes=params['hidden_layer_sizes'], random_state=42)
    gb_model.fit(X_train, y_train)

    gb_pred = gb_model.predict_proba(X_test)[:, 1]
    gb_roc_auc = roc_auc_score(y_test, gb_pred)
    print(params)
    print(gb_roc_auc)
    if (gb_roc_auc > bestScore): 
        bestParams = params
        bestScore = gb_roc_auc
print()
print(bestParams)
print(bestScore)

In [290]:
# Обучение MLP (Multi-Layer Perceptron) нейронной сети
#{'solver': 'lbfgs', 'max_iter': 900, 'alpha': 1e-07, 'hidden_layer_sizes': 19}
mlp_model = MLPClassifier(solver='lbfgs',alpha=1e-07,hidden_layer_sizes=(19,), max_iter=800, random_state=42)
mlp_model.fit(X_train, y_train)

# Прогноз на тестовых данных
mlp_pred = mlp_model.predict(X_test)
mlp_pred_prob = mlp_model.predict_proba(X_test)[:, 1]

# Расчет метрик для MLP нейронной сети
mlp_roc_auc = roc_auc_score(y_test, mlp_pred_prob)
mlp_accuracy = accuracy_score(y_test, mlp_pred)
mlp_precision = precision_score(y_test, mlp_pred)
mlp_recall = recall_score(y_test, mlp_pred)

print("\nMLP (Neural Network) метрики:")
print(f"ROC AUC: {mlp_roc_auc:.2f}")
print(f"Accuracy: {mlp_accuracy:.2f}")
print(f"Precision: {mlp_precision:.2f}")
print(f"Recall: {mlp_recall:.2f}")


MLP (Neural Network) метрики:
ROC AUC: 0.81
Accuracy: 0.77
Precision: 0.79
Recall: 0.91


## Экспериментируйте
Для получения лучшего качества придется поэкспериментировать. Подсказка: попробуйте оптимизировать гиперпараметры модели